# Setup

Install all needed dependencies.

In [1]:
!pip install --quiet --upgrade langchain_community langchain-huggingface langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codeflare-sdk 0.26.0 requires pydantic<2, but you have pydantic 2.11.1 which is incompatible.
kfp 2.9.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Install the YAML magic
!pip install --quiet yamlmagic
%load_ext yamlmagic


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Configuration and used models.

In [2]:
%%yaml parameters

# Model
embedder_model_name_or_path: ibm-granite/granite-embedding-30m-english
generator_model_name_or_path: ibm-granite/granite-3.2-2b-instruct

<IPython.core.display.Javascript object>

#  Langchain RAG

Prepare list of documents containing chunks.

In [3]:
import urllib.request
from langchain_core.documents import Document


link = "https://huggingface.co/ngxson/demo_simple_rag_py/raw/main/cat-facts.txt"
documents = []

# Retrieve knowledge from provided link, use every line as a separate chunk.
for line in urllib.request.urlopen(link):
    documents.append(
        Document(
            page_content=line.decode('utf-8'),
            metadata={"source": "cats", "doc_id": "cats"}
        )
    )

print(f'Loaded {len(documents)} entries')

Loaded 150 entries


Initialize vector store and fill it with chunks.

In [4]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(
    model_name=parameters['embedder_model_name_or_path'],
)
vector_store = InMemoryVectorStore(embeddings)
document_ids = vector_store.add_documents(documents=documents)

**Specify user query here**

In [5]:
input_query = "tell me about cat mummies"

Use similarity search to retrieve most related chunks from vector database.

In [6]:
docs = vector_store.similarity_search(input_query)

print('Retrieved knowledge:')
for doc in docs:
  print(doc)

Retrieved knowledge:
page_content='In ancient Egypt, mummies were made of cats, and embalmed mice were placed with them in their tombs. In one ancient city, over 300,000 cat mummies were found.
' metadata={'source': 'cats', 'doc_id': 'cats'}
page_content='In 1888, more than 300,000 mummified cats were found an Egyptian cemetery. They were stripped of their wrappings and carted off to be used by farmers in England and the U.S. for fertilizer.
' metadata={'source': 'cats', 'doc_id': 'cats'}
page_content='When a family cat died in ancient Egypt, family members would mourn by shaving off their eyebrows. They also held elaborate funerals during which they drank wine and beat their breasts. The cat was embalmed with a sculpted wooden mask and the tiny mummy was placed in the family tomb or in a pet cemetery with tiny mummies of mice.
' metadata={'source': 'cats', 'doc_id': 'cats'}
page_content='Mohammed loved cats and reportedly his favorite cat, Muezza, was a tabby. Legend says that tabby c

Generate response for user query using context from dataset.

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import transformers


llm = ChatOpenAI(
    model="test-model",
    openai_api_key="",
    openai_api_base="",
    max_tokens=1024,
    temperature=0,
)

tokenizer = transformers.AutoTokenizer.from_pretrained(parameters['generator_model_name_or_path'])
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Construct system prompt for inference providing retrieved chunks as context.
prompt = tokenizer.apply_chat_template(
    conversation=[{
        "role": "user",
        "content": "{input}",
    }],
    documents=[{
        "title": "placeholder",
        "text": "{context}",
    }],
    add_generation_prompt=True,
    tokenize=False,
)
prompt_template = PromptTemplate.from_template(template=prompt)

# Wrap retrieved chunks using prompt template
document_prompt_template = PromptTemplate.from_template(template="""\
Document {doc_id}
{page_content}""")
document_separator="\n\n"

# Create retrieval chain
combine_docs_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt_template,
    document_prompt=document_prompt_template,
    document_separator=document_separator,
)
rag_chain = create_retrieval_chain(
    retriever=vector_store.as_retriever(),
    combine_docs_chain=combine_docs_chain,
)

In [12]:
output = rag_chain.invoke({"input": input_query})

print(output['answer'])

Cat mummies have a significant historical and cultural presence, particularly in ancient Egypt. Here are some key points:

1. **Ancient Egyptian Practice**: In ancient Egypt, cats were mummified as part of their funeral rituals. The mummification process involved embalming the cat with a sculpted wooden mask, creating a tiny mummy.

2. **Embalmed Cats in Tomb and Pet Cemeteries**: The embalmed cat mummies were often placed in family tombs or in pet cemeteries, sometimes alongside tiny mummies of mice. This practice suggests a deep connection between cats and their owners, possibly symbolizing the cat's role in protecting the deceased in the afterlife.

3. **Mass Mummification**: In 1888, over 300,000 cat mummies were discovered in an Egyptian cemetery. These were stripped of their wrappings and exported to England and the U.S. for use as fertilizer, indicating the high value placed on cat mummies in ancient times.

4. **Cultural Significance**: The mourning rituals of ancient Egyptians